In [1]:
from treble_tsdk.tsdk import TSDK, TSDKCredentials
from treble_tsdk import display_data as dd
from treble_tsdk import treble
import random
import json
from tqdm import tqdm
import glob
import math
from typing import List, Tuple
from collections import defaultdict

In [6]:
data_dir = "data"

rooms_data = []
for filename in tqdm(glob.glob(f"{data_dir}/**/*.json", recursive=True), desc="Loading room data"):
    with open(filename, "r") as f:
        data = json.load(f)
        room_name = filename.split("/")[-2]
        data["room_name"] = room_name
        rooms_data.append(data)

tsdk = TSDK(TSDKCredentials.from_file("./creds/tsdk.cred"))


Loading room data: 100%|██████████| 10/10 [00:00<00:00, 13711.36it/s]


== SDK package is up to date ==


In [13]:
project = tsdk.get_or_create_project("ertsi_2")

In [8]:
def sort_points_clockwise(points: List[List[float]]) -> List[List[float]]:
    # Compute centroid
    centroid_x = sum(x for x, y in points) / len(points)
    centroid_y = sum(y for x, y in points) / len(points)
    
    # Function to compute angle from centroid
    def angle_from_centroid(point):
        x, y = point
        return math.atan2(y - centroid_y, x - centroid_x)
    
    # Sort points by angle (clockwise)
    return sorted(points, key=angle_from_centroid, reverse=True)

In [ ]:
room_definitions = []
for i, room_data in enumerate(rooms_data):
    edge_points_hexagon = sort_points_clockwise(room_data["room_verts"]) if not room_data["sorted"] else room_data["room_verts"]
    room_height = 3
    room = treble.GeometryDefinitionGenerator.create_polygon_room(
        points_xy=edge_points_hexagon, height_z=room_height, join_wall_layers=True
    )

    room_definitions.append(room)
room_definitions

[GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.,
 GeometryDefinition(geometry_component_count=0.]

In [15]:
generated_rooms = []

print("=== Populating rooms with geometry components ===")
for i, (room_def, room_data) in tqdm(enumerate(zip(room_definitions, rooms_data))):
    room_def.clear_geometry_components()
    placements = []

    if room_data["chair_count"] > 0:
        placements += [
            treble.GeometryComponentPlacement(
                components=tsdk.geometry_component_library.query(group="chair"),
            preferred_count=room_data["chair_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
            )
        ]
    if room_data["desk_count"] > 0:
        placements += [
            treble.GeometryComponentPlacement(
               components=tsdk.geometry_component_library.query(group="desk"),
            preferred_count=room_data["desk_count"],
            rotation_settings=treble.ComponentAnglePool([0, 90, 180, 270]),
            min_dist_from_objects=0.5,
            min_dist_from_walls=0.5,
            )
        ]
        
    room_def.populate_with_geometry_components(
        components=placements,
        selection_algorithm=treble.ComponentSelectionAlgorithm.random,
    )

    model = project.add_model(f"room_{i}", room_def)
    generated_rooms.append(model)

=== Populating rooms with geometry components ===


10it [00:27,  2.70s/it]


In [33]:
random_model = random.choice(generated_rooms)
random_model.plot()

In [21]:
dd.display(generated_rooms)

                                                 Models                                                 
┏━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Index ┃ Name   ┃ Id                                   ┃ Created At                   ┃ Is Watertight ┃
┡━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 0     │ room_0 │ b22dc5a4-5b65-4288-be77-7213d5ff9373 │ 2025-05-25 14:32:01          │ True          │
│ 1     │ room_1 │ 631a4285-fddd-412e-8549-42ab7bc946cb │ 2025-05-25 14:32:04          │ True          │
│ 2     │ room_2 │ 43e809ad-a2de-4a5e-95ad-2fcf965befc9 │ 2025-05-25 14:32:06          │ True          │
│ 3     │ room_3 │ e5eb3ea4-6d26-4261-8cc3-45cbc3b536b3 │ 2025-05-25 14:32:09          │ True          │
│ 4     │ room_4 │ 4be6e1e9-e574-46d9-b0b9-c8647b97c344 │ 2025-05-25T14:32:14.9253122Z │ False         │
│ 5     │ room_5 │ 28540add-e6fe-4332-8ef7-d45f2ca94e06 │ 2025-05-25T14:32:17.9302631Z │ False         │
│ 6     │ room_6 │ cd92839e-51d1-452c-acfc-9d43280ad2bf │ 2025-05-25T14:32:19.8537831Z │ False         │
│ 7     │ room_7 │ f3369126-6623-41fd-86e2-e244c1413c33 │ 2025-05-25T14:32:21.4806061Z │ False         │
│ 8     │ room_8 │ d1bc3ef4-4d1a-4bbc-8faf-4e4582c09d96 │ 2025-05-25T14:32:24.9603537Z │ False         │
│ 9     │ room_9 │ 476ebc9b-2a19-4007-9c1e-9345a0d1c588 │ 2025-05-25 14:32:27          │ False         │
└───────┴────────┴──────────────────────────────────────┴──────────────────────────────┴───────────────┘

In [34]:
all_materials = tsdk.material_library.get()
database_materials = [
    material for material in all_materials if material["organizationId"] == None
]

all_material_assignments = []
for i, (model, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    layers = {
        "polygon_room_walls": "gypsum/plaster on solid backing",
        "polygon_room_floor": room_data["floor_material"],
        "polygon_room_ceiling": room_data["ceiling_material"],
        "Furniture/Desk": "wood",
        "Furniture/Chair": "upholstered concert chairs",
        "Furniture/Chair A": "upholstered concert chairs",
        "Furniture/Bar Stool": "upholstered concert chairs",
    }
    material_assignment = []

    for layer in model.layer_names:
        if layer in layers:
            search_string = layers[layer]
            matches = [
                m for m in database_materials if search_string.lower() in m.name.lower()
            ]
            if matches:
                material_assignment.append(
                    treble.MaterialAssignment(layer, random.choice(matches))
                )
    all_material_assignments.append(material_assignment)

dd.display(all_material_assignments[-1])

10it [00:00, 90.19it/s]


                                   Material assignment                                    
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer name           ┃ Material name                                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ polygon_room_ceiling │ Slotted panel, 21% open, 40 mm 81 kg/m3 absorber, 100 mm cavity │
│ Furniture/Chair      │ Heavily upholstered concert chairs                              │
│ Furniture/Bar Stool  │ Heavily upholstered concert chairs                              │
│ Furniture/Desk       │ Plywood panelling, 1 cm thick                                   │
│ Furniture/Chair A    │ Upholstered concert chairs                                      │
│ polygon_room_floor   │ Linoleum or vinyl on concrete                                   │
│ polygon_room_walls   │ Gypsum/Plaster on solid backing                                 │
└──────────────────────┴─────────────────────────────────────────────────────────────────┘

In [35]:
all_sources = []
all_source_positions = []
all_receivers = []
all_receiver_positions = []

pg = treble.PointsGenerator()

pos_ruleset = treble.PointRuleset(
    min_dist_from_surface=0.5,
    min_dist_from_other_points=2,
)

for room in tqdm(generated_rooms):
    pos = pg.generate_valid_points(
        model=room,
        max_count=2,	
        ruleset=pos_ruleset,
        z_range=(0.5, 1.5)
    )
    all_source_positions.append(pos[0])
    all_receiver_positions.append(pos[1])

    source = treble.Source.make_omni(
        position=pos[0],
        label=f"source_{room.name}",
    )
    receiver = treble.Receiver.make_mono(
        position=pos[1],
        label=f"receiver_{room.name}",
    )
    all_sources.append(source)
    all_receivers.append(receiver)


100%|██████████| 10/10 [00:01<00:00,  5.59it/s]


In [ ]:
sim_type = treble.SimulationType.hybrid
crossover_frequency = 500

sim_defs = []

for i, (room, room_data) in tqdm(enumerate(zip(generated_rooms, rooms_data))):
    sim_def = treble.SimulationDefinition(
        name=f"simulation_{i}_1",  # unique name of the simulation
        simulation_type=sim_type,  # the type of simulation
        crossover_frequency=crossover_frequency,  # the frequency at which the simulation switches from wavebased to image source
        model=room,
        energy_decay_threshold=60,  # simulation termination criteria - the simulation stops running after -60 dB of energy decay
        receiver_list=[all_receivers[i]],
        source_list=[all_sources[i]],
        material_assignment=all_material_assignments[i],
    )
    sim_defs.append(sim_def)

sim_defs = project.add_simulations(sim_defs)

10it [00:00, 38657.18it/s]
One or more validation errors occurred.
Field: "ValidationError" - ['Duplicate simulation names in project found: simulation_0_1,simulation_1_1,simulation_2_1,simulation_3_1,simulation_4_1 please use unique simulation names.']


In [37]:
all_sims = project.get_simulations()
dd.display(all_sims)

                                                      Simulations                                                      
┏━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━┳━━━━━┓
┃ Index ┃ Name           ┃ Created at       ┃ Id                                   ┃ Status  ┃ Freq. (Hz) ┃ Rcv ┃ Src ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━╇━━━━━┩
│ 0     │ simulation_0_1 │ 2025-05-25 14:34 │ cddbbca2-6ef0-4ccd-9b64-2324ba58b3a7 │ Created │ 500        │ 1   │ 1   │
│ 1     │ simulation_2_1 │ 2025-05-25 14:34 │ 2c4f189e-26a0-4506-9ee7-2b76374a44b2 │ Created │ 500        │ 1   │ 1   │
│ 2     │ simulation_3_1 │ 2025-05-25 14:34 │ 4bbf54b1-ba0c-4e23-a7d2-172b6a51befb │ Created │ 500        │ 1   │ 1   │
│ 3     │ simulation_1_1 │ 2025-05-25 14:34 │ dc8daa95-29de-4621-988f-081c9e538b02 │ Created │ 500        │ 1   │ 1   │
│ 4     │ simulation_4_1 │ 2025-05-25 14:34 │ e264eaba-4a8b-4147-9923-554a0333aecd │ Created │ 500        │ 1   │ 1   │
│ 5     │ simulation_8_1 │ 2025-05-25 14:34 │ 076a1fc8-4b67-43bf-9ecf-8eb5d58d775a │ Created │ 500        │ 1   │ 1   │
│ 6     │ simulation_5_1 │ 2025-05-25 14:34 │ 8bae7a7a-20c6-4760-a364-c8f2ca8bda96 │ Created │ 500        │ 1   │ 1   │
│ 7     │ simulation_7_1 │ 2025-05-25 14:34 │ 8cdc31ce-2d67-47f0-aad3-fdd7efe0ba56 │ Created │ 500        │ 1   │ 1   │
│ 8     │ simulation_6_1 │ 2025-05-25 14:34 │ fb3af502-535c-48f7-83f5-b46956854b27 │ Created │ 500        │ 1   │ 1   │
│ 9     │ simulation_9_1 │ 2025-05-25 14:34 │ b44ebd64-9a1f-472e-8a7c-493068cae254 │ Created │ 500        │ 1   │ 1   │
└───────┴────────────────┴──────────────────┴──────────────────────────────────────┴─────────┴────────────┴─────┴─────┘

In [41]:
random_sim = random.choice(all_sims)
random_sim.plot()

In [42]:
runtime_estimate = project.estimate()
dd.display(runtime_estimate)


                               Project 8f3ea3d4-3701-4030-b97d-949ac2b566e3 estimate                               
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃                                                       ┃       Simulation ┃ Estimated runtime ┃   Estimated cost ┃
┃                                            Simulation ┃           status ┃           (hours) ┃         (tokens) ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ simulation_0_1 (cddbbca2-6ef0-4ccd-9b64-2324ba58b3a7) │          Created │             0.074 │            0.103 │
│ simulation_2_1 (2c4f189e-26a0-4506-9ee7-2b76374a44b2) │          Created │             0.262 │            0.307 │
│ simulation_3_1 (4bbf54b1-ba0c-4e23-a7d2-172b6a51befb) │          Created │             0.172 │            0.207 │
│ simulation_1_1 (dc8daa95-29de-4621-988f-081c9e538b02) │          Created │             0.058 │            0.072 │
│ simulation_4_1 (e264eaba-4a8b-4147-9923-554a0333aecd) │          Created │             0.261 │            0.289 │
│ simulation_8_1 (076a1fc8-4b67-43bf-9ecf-8eb5d58d775a) │          Created │             0.064 │            0.078 │
│ simulation_5_1 (8bae7a7a-20c6-4760-a364-c8f2ca8bda96) │          Created │             0.038 │             0.05 │
│ simulation_7_1 (8cdc31ce-2d67-47f0-aad3-fdd7efe0ba56) │          Created │             0.101 │            0.118 │
│ simulation_6_1 (fb3af502-535c-48f7-83f5-b46956854b27) │          Created │             0.105 │            0.139 │
│ simulation_9_1 (b44ebd64-9a1f-472e-8a7c-493068cae254) │          Created │             0.046 │            0.064 │
├───────────────────────────────────────────────────────┼──────────────────┼───────────────────┼──────────────────┤
│                                                 Total │                  │             0.262 │            1.427 │
└───────────────────────────────────────────────────────┴──────────────────┴───────────────────┴──────────────────┘

In [44]:
project.start_simulations()
project.as_live_progress()

                                      ertsi_2                                      
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Simulation Name ┃ Id                                   ┃ Status    ┃ Progress % ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ simulation_8_1  │ 076a1fc8-4b67-43bf-9ecf-8eb5d58d775a │ Completed │ 100%       │
│ simulation_2_1  │ 2c4f189e-26a0-4506-9ee7-2b76374a44b2 │ Completed │ 100%       │
│ simulation_3_1  │ 4bbf54b1-ba0c-4e23-a7d2-172b6a51befb │ Completed │ 100%       │
│ simulation_5_1  │ 8bae7a7a-20c6-4760-a364-c8f2ca8bda96 │ Completed │ 100%       │
│ simulation_7_1  │ 8cdc31ce-2d67-47f0-aad3-fdd7efe0ba56 │ Completed │ 100%       │
│ simulation_9_1  │ b44ebd64-9a1f-472e-8a7c-493068cae254 │ Completed │ 100%       │
│ simulation_0_1  │ cddbbca2-6ef0-4ccd-9b64-2324ba58b3a7 │ Completed │ 100%       │
│ simulation_1_1  │ dc8daa95-29de-4621-988f-081c9e538b02 │ Completed │ 100%       │
│ simulation_4_1  │ e264eaba-4a8b-4147-9923-554a0333aecd │ Completed │ 102.44%    │
│ simulation_6_1  │ fb3af502-535c-48f7-83f5-b46956854b27 │ Completed │ 100%       │
├─────────────────┼──────────────────────────────────────┼───────────┼────────────┤
│                 │                                      │ Completed │ 100.24%    │
└─────────────────┴──────────────────────────────────────┴───────────┴────────────┘

Done

In [66]:
for sim, room in zip(all_sims, rooms_data):
    room_name = room["room_name"]
    
    try:
        r = sim.download_results(f'data/{room_name}')
    except ValueError as e:
        print(f"Room {room_name} already exists, skipping download.")
        r = None

Room leo_2 already exists, skipping download.
Room leo_4 already exists, skipping download.
Room leo_10 already exists, skipping download.
Room leo_8 already exists, skipping download.
Room leo_7 already exists, skipping download.
Room leo_6 already exists, skipping download.
Room leo_1 already exists, skipping download.
Room leo_5 already exists, skipping download.
Room leo_9 already exists, skipping download.
Room leo_3 already exists, skipping download.


In [71]:
for simulation, room in zip(all_sims, rooms_data):
    try:
        results = simulation.get_results_object(f'data/{room["room_name"]}')
        mono_ir = results.get_mono_ir(source=simulation.sources[0], receiver=simulation.receivers[0])
        mono_ir.write_to_wav(path_to_file=f"data/{room['room_name']}/ir.wav")
    except KeyError as e:
        print(f"Results not found for room {room['room_name']}: {e}")


Results not found for room leo_7: 'Unable to synchronously open object (component not found)'
